<div dir="rtl">


* چگونه تعادل برای مجموعه داده‌های نامتعادل انجام می‌شود
* چگونه رمزگذاری برچسب برای متغیرهای دسته‌ای انجام می‌شود
* چگونه روش‌های پیشرفته مانند MICE برای تکمیل داده‌ها استفاده می‌شوند
* چگونه می‌توان نقاط ناگوار را شناسایی کرده و از داده‌ها حذف کرد
* چگونه روش‌های فیلتر و پوشش برای انتخاب ویژگی‌ها استفاده می‌شوند
* چگونه می‌توان مقایسه کرد تعادل سرعت و عملکرد برای مدل‌های محبوب مختلف
* کدام معیار می‌تواند بهترین معیار برای ارزیابی عملکرد در مجموعه داده‌های نامتعادل باشد: دقت (Accuracy)، اسکور F1 یا کاپا کوئن (Cohen's Kappa)

<div dir="rtl">


# وارد کردن داده‌ها

In [ ]:
import pandas as pd
full_data = pd.read_csv('weatherAUS.csv')
full_data.head()

<div dir="rtl">


# بررسی داده‌ها
ابتدا تعداد ردیف‌ها و ستون‌ها را بررسی خواهیم کرد. سپس اندازه مجموعه داده را برای تصمیم‌گیری در مورد نیاز به فشرده‌سازی اندازه آن بررسی خواهیم کرد.

In [ ]:
full_data.shape

In [ ]:
full_data.info()

<div dir="rtl">

هر دو "RainToday" و "RainTomorrow" اشیاء هستند (بله/خیر). ما آن‌ها را به صورت دودویی (1/0) برای راحتی خود تبدیل خواهیم کرد.

In [ ]:
full_data['RainToday'].replace({'No': 0, 'Yes': 1},inplace = True)
full_data['RainTomorrow'].replace({'No': 0, 'Yes': 1},inplace = True)

<div dir="rtl">بعداً، ما بررسی خواهیم کرد که آیا مجموعه داده ناهمواره است یا متوازن. اگر مجموعه داده ناهموار باشد، ما نیاز داریم که تعداد نمونه‌های اکثریت را کاهش دهیم یا تعداد نمونه‌های اقلیت را افزایش دهیم تا آن را متوازن کنیم.

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (8,5))
full_data.RainTomorrow.value_counts(normalize = True).plot(kind='bar', color= ['skyblue','navy'], alpha = 0.9, rot=0)
plt.title('RainTomorrow Indicator No(0) and Yes(1) in the Imbalanced Dataset')
plt.show()

<div dir="rtl">

می‌توانیم مشاهده کنیم که حضور "0" و "1" تقریباً به نسبت ۷۸ به ۲ است. بنابراین تراز کلاس وجود دارد و ما باید با آن برخورد کنیم. برای مقابله با تراز کلاس، ما در اینجا از **افزایش تعداد نمونه‌های کلاس اقلیت** استفاده خواهیم کرد. از آنجا که اندازه مجموعه داده به نسبت کوچک است، کاهش تعداد نمونه‌های کلاس اکثریت در اینجا معنی زیادی نخواهد داشت.

<div dir="rtl">

# مدیریت عدم توازن کلاس‌ها

In [ ]:
from sklearn.utils import resample

no = full_data[full_data.RainTomorrow == 0]
yes = full_data[full_data.RainTomorrow == 1]
yes_oversampled = resample(yes, replace=True, n_samples=len(no), random_state=123)
oversampled = pd.concat([no, yes_oversampled])

fig = plt.figure(figsize = (8,5))
oversampled.RainTomorrow.value_counts(normalize = True).plot(kind='bar', color= ['skyblue','navy'], alpha = 0.9, rot=0)
plt.title('RainTomorrow Indicator No(0) and Yes(1) after Oversampling (Balanced Dataset)')
plt.show()

<div dir="rtl">حالا، ما اکنون الگوی داده‌های از دست رفته را در مجموعه داده بررسی خواهیم کرد.

In [ ]:
# Missing Data Pattern in Training Data
import seaborn as sns
sns.heatmap(oversampled.isnull(), cbar=False, cmap='PuBu')

<div dir="rtl">

بطور قابل‌مشاهده، 'تبخیر'، 'تابش نور خورشید'، 'ابرهای ساعت 9 صبح' و 'ابرهای ساعت 3 بعد از ظهر' ویژگی‌هایی هستند که درصد بالایی از داده‌ها در آن‌ها از دست رفته است. بنابراین، ما جزئیات داده‌های از دست رفته برای این 4 ویژگی را بررسی خواهیم کرد.

In [ ]:
total = oversampled.isnull().sum().sort_values(ascending=False)
percent = (oversampled.isnull().sum()/oversampled.isnull().count()).sort_values(ascending=False)
missing = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing.head(4)

<div dir="rtl">ما مشاهده می‌کنیم که همه چهار ویژگی دارای داده‌های گم‌شده کمتر از ۵۰٪ هستند. بنابراین به جای کاملاً حذف آن‌ها، ما آن‌ها را با روش‌های مناسب در مدل خود در نظر خواهیم گرفت.

<div dir="rtl">

# تکمیل و تبدیل داده
در این مرحله، ستون‌های دسته‌ای را با استفاده از مد تکمیل کرده و سپس از رمزگذار برچسبی برای تبدیل آن‌ها به متغیرهای عددی استفاده می‌کنیم. بعد از آن، به کمک بسته MICE (Multiple Imputation by Chained Equations)، مقادیر گم‌شده (مانند NaN و غیره) را تکمیل می‌کنیم. سپس، با استفاده از محدوده بین‌کارها (Inter-Quartile Range)، نقاط نادرست را شناسایی کرده و آن‌ها را حذف کرده و به مجموعه داده نهایی می‌رسیم. در نهایت، ارتباط بین متغیرهای مختلف را بررسی کرده و اگر هر گونه جفت متغیر با ارتباط بالایی پیدا کنیم، یکی از آن‌ها را حذف کرده و دیگری را حفظ می‌کنیم.


In [ ]:
oversampled.select_dtypes(include=['object']).columns

In [ ]:
# Impute categorical var with Mode
oversampled['Date'] = oversampled['Date'].fillna(oversampled['Date'].mode()[0])
oversampled['Location'] = oversampled['Location'].fillna(oversampled['Location'].mode()[0])
oversampled['WindGustDir'] = oversampled['WindGustDir'].fillna(oversampled['WindGustDir'].mode()[0])
oversampled['WindDir9am'] = oversampled['WindDir9am'].fillna(oversampled['WindDir9am'].mode()[0])
oversampled['WindDir3pm'] = oversampled['WindDir3pm'].fillna(oversampled['WindDir3pm'].mode()[0])

In [ ]:
# Convert categorical features to continuous features with Label Encoding
from sklearn.preprocessing import LabelEncoder
lencoders = {}
for col in oversampled.select_dtypes(include=['object']).columns:
    lencoders[col] = LabelEncoder()
    oversampled[col] = lencoders[col].fit_transform(oversampled[col])

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Multiple Imputation by Chained Equations
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
MiceImputed = oversampled.copy(deep=True) 
mice_imputer = IterativeImputer()
MiceImputed.iloc[:, :] = mice_imputer.fit_transform(oversampled)

In [ ]:
MiceImputed.head()

<div dir="rtl">حالا، ما بررسی می‌کنیم که آیا تمام مقادیر "NaN" به طور کامل ترمیم شده‌اند یا خیر.

In [ ]:
MiceImputed.isna()

<div dir="rtl">پس از تخمین با MICE، دیتافریم دیگر هیچ مقدار "NaN" ندارد. اکنون ما قصد داریم نویزها را از مجموعه داده بر اساس دامنه میانکاره رد کنیم.

In [ ]:
# Detecting outliers with IQR
Q1 = MiceImputed.quantile(0.25)
Q3 = MiceImputed.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# Removing outliers from dataset
MiceImputed = MiceImputed[~((MiceImputed < (Q1 - 1.5 * IQR)) |(MiceImputed > (Q3 + 1.5 * IQR))).any(axis=1)]
MiceImputed.shape

In [ ]:
# Correlation Heatmap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
corr = MiceImputed.corr()
mask = np.triu(np.ones_like(corr))
f, ax = plt.subplots(figsize=(20, 20))
cmap = sns.diverging_palette(250, 25, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=None, center=0,square=True, annot=True, linewidths=.5, cbar_kws={"shrink": .9})

<div dir="rtl">

در زیر، جفت‌های ویژگی‌هایی که بین آن‌ها همبستگی بالایی وجود دارد آمده است:

* حداکثر دما و حداقل دما
* فشار ساعت 9 صبح و فشار ساعت 3 بعد از ظهر
* دما ساعت 9 صبح و دما ساعت 3 بعد از ظهر
* تبخیر و حداکثر دما
* حداکثر دما و دما ساعت 3 بعد از ظهر

اما در هیچ یک از موارد، مقدار همبستگی به صورت کاملاً "1" نیست. بنابراین هیچ ویژگی را حذف نمی‌کنیم.

با این حال، می‌توانیم به عمق در همبستگی جفتی بین این ویژگی‌های با همبستگی بالا نگاه کنیم، با توجه به نمودار جفتی زیر. هر یک از نمودارهای جفتی به وضوح خوشه‌های قابل تمییزی از "بله" و "خیر" برای باران فردا نشان می‌دهد. همگی دارای اشتراک بسیار کمی هستند.


In [ ]:
sns.pairplot( data=MiceImputed, vars=('MaxTemp','MinTemp','Pressure9am','Pressure3pm', 'Temp9am', 'Temp3pm', 'Evaporation'), hue='RainTomorrow' )


<div dir="rtl">

# انتخاب ویژگی
برای انتخاب ویژگی، از روش فیلتر و روش پوشش‌دهنده استفاده خواهیم کرد.

**(1) انتخاب ویژگی با استفاده از روش فیلتر (مقدار کای-مربع):** قبل از انجام این کار، نیاز داریم که داده‌های خود را استاندارد کنیم. ما از MinMaxScaler به جای StandardScaler استفاده می‌کنیم تا از ایجاد مقادیر منفی جلوگیری کنیم.

In [ ]:
# Standardizing data
from sklearn import preprocessing
r_scaler = preprocessing.MinMaxScaler()
r_scaler.fit(MiceImputed)
modified_data = pd.DataFrame(r_scaler.transform(MiceImputed), index=MiceImputed.index, columns=MiceImputed.columns)
modified_data.head()

In [ ]:
# Feature Importance using Filter Method (Chi-Square)
from sklearn.feature_selection import SelectKBest, chi2
X = modified_data.loc[:,modified_data.columns!='RainTomorrow']
y = modified_data[['RainTomorrow']]
selector = SelectKBest(chi2, k=10)
selector.fit(X, y)
X_new = selector.transform(X)
print(X.columns[selector.get_support(indices=True)])

<div dir="rtl">


می‌توانیم مشاهده کنیم که "Sunshine"، "Humidity9am"، "Humidity3pm"، "Pressure9am" و "Pressure3pm" اهمیت بالاتری نسبت به ویژگی‌های دیگر دارند.

**(2) انتخاب ویژگی با استفاده از روش Wrapper (جنگل تصادفی):**

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier as rf

X = MiceImputed.drop('RainTomorrow', axis=1)
y = MiceImputed['RainTomorrow']
selector = SelectFromModel(rf(n_estimators=100, random_state=0))
selector.fit(X, y)
support = selector.get_support()
features = X.loc[:,support].columns.tolist()
print(features)
print(rf(n_estimators=100, random_state=0).fit(X,y).feature_importances_)

<div dir="rtl">


این خیلی جالب است که تمام اهمیت‌های ویژگی به جز RISK_MM به نزدیک صفر می‌رسند. این در دو حالت ممکن است رخ دهد. یا زمانی که تمام ویژگی‌ها دارای همبستگی بالا با یکدیگر باشند، یا زمانی که ویژگی‌ها در واقع اهمیت نسبی بسیار پایینی نسبت به متغیر هدف داشته باشند. از آنجا که ما در حالت قبلی همبستگی را رسم کرده‌ایم، می‌دانیم که احتمال اول درست نیست. ما با استفاده از اهمیت جابجایی (Permutation Importance) بررسی می‌کنیم که آیا احتمال دوم درست است یا خیر.

In [ ]:
import sklearn
print(sklearn.__version__)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


In [ ]:
ConfusionMatrixDisplay.from_estimator(model, X_test, y_test)


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rf(n_estimators=100, random_state=0).fit(X,y),random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

<div dir="rtl">

# آموزش با مدل‌های مختلف
ما کل مجموعه داده را به دو بخش آموزش (75%) و آزمایش (25%) تقسیم می‌کنیم. برای به دست آوردن نتایج بهتر، داده‌های X_train و X_test خود را استاندارد می‌کنیم (یعنی ویژگی‌ها بدون هدف برای مجموعه‌های داده آموزش و آزمایش).

In [ ]:
features = MiceImputed[['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 
                       'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 
                       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 
                       'RainToday']]
target = MiceImputed['RainTomorrow']

# Split into test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=12345)

# Normalize Features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
def plot_roc_cur(fper, tper):  
    plt.plot(fper, tper, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

In [ ]:
import time
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score, ConfusionMatrixDisplay, roc_curve, classification_report
def run_model(model, X_train, y_train, X_test, y_test, verbose=True):
    t0=time.time()
    if verbose == False:
        model.fit(X_train,y_train, verbose=0)
    else:
        model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred) 
    coh_kap = cohen_kappa_score(y_test, y_pred)
    time_taken = time.time()-t0
    print("Accuracy = {}".format(accuracy))
    print("ROC Area under Curve = {}".format(roc_auc))
    print("Cohen's Kappa = {}".format(coh_kap))
    print("Time taken = {}".format(time_taken))
    print(classification_report(y_test,y_pred,digits=5))
    
    probs = model.predict_proba(X_test)  
    probs = probs[:, 1]  
    fper, tper, thresholds = roc_curve(y_test, probs) 
    plot_roc_cur(fper, tper)
    
    ConfusionMatrixDisplay.from_estimator(model, X_test, y_test)
    
    return model, accuracy, roc_auc, coh_kap, time_taken

<div dir="rtl">

**مدل ۱: رگرسیون لجستیک تنظیم شده توسط لسو**

In [ ]:
from sklearn.linear_model import LogisticRegression

params_lr = {'penalty': 'l1', 'solver':'liblinear'}

model_lr = LogisticRegression(**params_lr)
model_lr, accuracy_lr, roc_auc_lr, coh_kap_lr, tt_lr = run_model(model_lr, X_train, y_train, X_test, y_test)

<div dir="rtl">

**مدل-2: درخت تصمیم**

In [ ]:
from sklearn.tree import DecisionTreeClassifier

params_dt = {'max_depth': 16,
             'max_features': "sqrt"}

model_dt = DecisionTreeClassifier(**params_dt)
model_dt, accuracy_dt, roc_auc_dt, coh_kap_dt, tt_dt = run_model(model_dt, X_train, y_train, X_test, y_test)

<div dir="rtl">

**مدل-3: شبکه عصبی (مولتی لایه پرسپترون)**

In [ ]:
from sklearn.neural_network import MLPClassifier

params_nn = {'hidden_layer_sizes': (30,30,30),
             'activation': 'logistic',
             'solver': 'lbfgs',
             'max_iter': 500}

model_nn = MLPClassifier(**params_nn)
model_nn, accuracy_nn, roc_auc_nn, coh_kap_nn, tt_nn = run_model(model_nn, X_train, y_train, X_test, y_test)

<div dir="rtl">

**مدل-۴: جنگل تصادفی**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

params_rf = {'max_depth': 16,
             'min_samples_leaf': 1,
             'min_samples_split': 2,
             'n_estimators': 100,
             'random_state': 12345}

model_rf = RandomForestClassifier(**params_rf)
model_rf, accuracy_rf, roc_auc_rf, coh_kap_rf, tt_rf = run_model(model_rf, X_train, y_train, X_test, y_test)

<div dir="rtl">


**مدل-۵: Light GBM**

In [ ]:
import lightgbm as lgb
params_lgb ={'colsample_bytree': 0.95, 
         'max_depth': 16, 
         'min_split_gain': 0.1, 
         'n_estimators': 200, 
         'num_leaves': 50, 
         'reg_alpha': 1.2, 
         'reg_lambda': 1.2, 
         'subsample': 0.95, 
         'subsample_freq': 20}

model_lgb = lgb.LGBMClassifier(**params_lgb)
model_lgb, accuracy_lgb, roc_auc_lgb, coh_kap_lgb, tt_lgb = run_model(model_lgb, X_train, y_train, X_test, y_test)

<div dir="rtl">

**مدل-6: CatBoost**

In [ ]:
import catboost as cb
params_cb ={'iterations': 50,
            'max_depth': 16}

model_cb = cb.CatBoostClassifier(**params_cb)
model_cb, accuracy_cb, roc_auc_cb, coh_kap_cb, tt_cb = run_model(model_cb, X_train, y_train, X_test, y_test, verbose=False)

<div dir="rtl">

**مدل-7: XGBoost**

In [ ]:
import xgboost as xgb
params_xgb ={'n_estimators': 500,
            'max_depth': 16}

model_xgb = xgb.XGBClassifier(**params_xgb)
model_xgb, accuracy_xgb, roc_auc_xgb, coh_kap_xgb, tt_xgb = run_model(model_xgb, X_train, y_train, X_test, y_test)

<div dir="rtl">

# ترسیم منطق تصمیم برای تمام مدل‌ها

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from mlxtend.classifier import EnsembleVoteClassifier
from mlxtend.plotting import plot_decision_regions

value = 1.80
width = 0.90

clf1 = LogisticRegression(random_state=12345)
clf2 = DecisionTreeClassifier(random_state=12345) 
clf3 = MLPClassifier(random_state=12345, verbose = 0)
clf4 = RandomForestClassifier(random_state=12345)
clf5 = lgb.LGBMClassifier(random_state=12345, verbose = 0)
clf6 = cb.CatBoostClassifier(random_state=12345, verbose = 0)
clf7 = xgb.XGBClassifier(random_state=12345)
eclf = EnsembleVoteClassifier(clfs=[clf4, clf5, clf6, clf7], weights=[1, 1, 1, 1], voting='soft')

X_list = MiceImputed[["Sunshine", "Humidity9am", "Cloud3pm"]] #took only really important features
X = np.asarray(X_list, dtype=np.float32)
y_list = MiceImputed["RainTomorrow"]
y = np.asarray(y_list, dtype=np.int32)

# Plotting Decision Regions
gs = gridspec.GridSpec(3,3)
fig = plt.figure(figsize=(18, 14))

labels = ['Logistic Regression',
          'Decision Tree',
          'Neural Network',
          'Random Forest',
          'LightGBM',
          'CatBoost',
          'XGBoost',
          'Ensemble']

for clf, lab, grd in zip([clf1, clf2, clf3, clf4, clf5, clf6, clf7, eclf],
                         labels,
                         itertools.product([0, 1, 2],
                         repeat=2)):
    clf.fit(X, y)
    ax = plt.subplot(gs[grd[0], grd[1]])
    fig = plot_decision_regions(X=X, y=y, clf=clf, 
                                filler_feature_values={2: value}, 
                                filler_feature_ranges={2: width}, 
                                legend=2)
    plt.title(lab)

plt.show()

<div dir="rtl">


می‌توانیم تفاوت در مرزهای کلاس برای مدل‌های مختلف را مشاهده کنیم، از جمله مدل *انسمبل* (نمودار‌ها با در نظر گرفتن داده‌های آموزش انجام شده است). CatBoost مرز منطقه‌ای متمایزی نسبت به تمام مدل‌های دیگر دارد. با این حال، مدل‌های XGBoost و Random Forest نیز تعداد بسیار کمتری از نقاط داده‌های اشتباه طبقه‌بندی‌شده را نسبت به مدل‌های دیگر دارند.

# مقایسه مدل

حالا باید تصمیم بگیریم که کدام مدل بر اساس امتیاز دقت، ROC_AUC، کوهن's کاپا و زمان کلی اجرا بهترین عملکرد را داشته است. نکته‌ای که در اینجا قابل توجه است: می‌توانستیم F1-Score را به عنوان معیار بهتری برای ارزیابی عملکرد مدل در نظر بگیریم تا دقت، اما ما قبلاً مجموعه داده‌های نامتعادل را به یک مجموعه داده متعادل تبدیل کرده‌ایم، بنابراین در نظر گرفتن دقت به عنوان یک معیار برای تصمیم‌گیری در مورد بهترین مدل در این مورد توجیه می‌شود. برای تصمیم‌گیری بهتر، ما **"کوهن's کاپا"** را انتخاب کرده‌ایم که در واقع یک انتخاب ایده‌آل به عنوان معیار برای تصمیم‌گیری در مورد مجموعه‌های داده نامتعادل است. بیایید بررسی کنیم که کدام مدل در کدام جنبه‌ها بهتر عمل کرده است.

In [ ]:
accuracy_scores = [accuracy_lr, accuracy_dt, accuracy_nn, accuracy_rf, accuracy_lgb, accuracy_cb, accuracy_xgb]
roc_auc_scores = [roc_auc_lr, roc_auc_dt, roc_auc_nn, roc_auc_rf, roc_auc_lgb, roc_auc_cb, roc_auc_xgb]
coh_kap_scores = [coh_kap_lr, coh_kap_dt, coh_kap_nn, coh_kap_rf, coh_kap_lgb, coh_kap_cb, coh_kap_xgb]
tt = [tt_lr, tt_dt, tt_nn, tt_rf, tt_lgb, tt_cb, tt_xgb]

model_data = {'Model': ['Logistic Regression','Decision Tree','Neural Network','Random Forest','LightGBM','Catboost','XGBoost'],
              'Accuracy': accuracy_scores,
              'ROC_AUC': roc_auc_scores,
              'Cohen_Kappa': coh_kap_scores,
              'Time taken': tt}
data = pd.DataFrame(model_data)

fig, ax1 = plt.subplots(figsize=(12,10))
ax1.set_title('Model Comparison: Accuracy and Time taken for execution', fontsize=13)
color = 'tab:green'
ax1.set_xlabel('Model', fontsize=13)
ax1.set_ylabel('Time taken', fontsize=13, color=color)
ax2 = sns.barplot(x='Model', y='Time taken', data = data, palette='summer')
ax1.tick_params(axis='y')
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Accuracy', fontsize=13, color=color)
ax2 = sns.lineplot(x='Model', y='Accuracy', data = data, sort=False, color=color)
ax2.tick_params(axis='y', color=color)

In [ ]:
fig, ax3 = plt.subplots(figsize=(12,10))
ax3.set_title('Model Comparison: Area under ROC and Cohens Kappa', fontsize=13)
color = 'tab:blue'
ax3.set_xlabel('Model', fontsize=13)
ax3.set_ylabel('ROC_AUC', fontsize=13, color=color)
ax4 = sns.barplot(x='Model', y='ROC_AUC', data = data, palette='winter')
ax3.tick_params(axis='y')
ax4 = ax3.twinx()
color = 'tab:red'
ax4.set_ylabel('Cohen_Kappa', fontsize=13, color=color)
ax4 = sns.lineplot(x='Model', y='Cohen_Kappa', data = data, sort=False, color=color)
ax4.tick_params(axis='y', color=color)
plt.show()

<div dir="rtl">

# نتیجه‌گیری
می‌توانیم مشاهده کنیم که **XGBoost، CatBoost و Random Forest** به مقایسه با مدل‌های دیگر بهتر عمل کرده‌اند. با این حال، اگر سرعت یک ملاحظه مهم باشد، می‌توانیم به جای XGBoost یا CatBoost به Random Forest پایبند باشیم.